In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from itertools import product
from numba import njit

def DataFetcher(symbol, interval='1d'):
    suffixes = ['.NS', '.BO']
    for suffix in suffixes:
        try:
            data = yf.download(symbol + suffix, interval=interval, progress=False)
            data.drop(columns='Adj Close', inplace=True)
            data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
            data['Time'] = data.index
            return data,symbol + suffix
        except Exception as e:
            #print(f"Failed to fetch data for {symbol+suffix}: {e}")
            continue
    return pd.DataFrame()


@njit
def calculate_supertrend_fast(high, low, close, volume, period, multiplier):
    atr = np.zeros_like(close)
    supertrend = np.zeros_like(close)
    direction = np.zeros_like(close)
    
    # Calculate ATR
    tr = np.maximum(high - low, np.abs(high - np.roll(close, 1)), np.abs(low - np.roll(close, 1)))
    atr[period-1] = np.mean(tr[:period])
    for i in range(period, len(close)):
        atr[i] = (atr[i-1] * (period - 1) + tr[i]) / period
    
    # Calculate volume factor
    volume_ma = np.zeros_like(volume)
    volume_ma[period-1] = np.mean(volume[:period])
    for i in range(period, len(volume)):
        volume_ma[i] = (volume_ma[i-1] * (period - 1) + volume[i]) / period
    volume_factor = volume / volume_ma
    
    # Calculate SuperTrend
    hl2 = (high + low) / 2
    basic_upperband = hl2 + (multiplier * atr * volume_factor)
    basic_lowerband = hl2 - (multiplier * atr * volume_factor)
    
    for i in range(period, len(close)):
        if close[i] > supertrend[i-1]:
            supertrend[i] = max(basic_lowerband[i], supertrend[i-1])
        else:
            supertrend[i] = min(basic_upperband[i], supertrend[i-1])
        
        if close[i] > supertrend[i]:
            direction[i] = 1
        else:
            direction[i] = -1

    return supertrend, direction

def calculate_super_trend(df, multiplier, period):
    supertrend, direction = calculate_supertrend_fast(
        df['high'].values, df['low'].values, df['close'].values, 
        df['volume'].values, period, multiplier
    )
    
    df['super_trend'] = supertrend
    df['signal'] = direction
    df['Position'] = df['signal'].diff()
    df['Buy/Sell'] = df['Position'].map({2: 'Buy', -2: 'Sell', 0: 'Hold'})
    df['VSMA'] = df['volume'].rolling(window=period).mean()
    
    return df




def calculate_super_trend_performance(df):
    if 'signal' not in df.columns:
      raise ValueError("Signal column not created. Check calculate_super_trend function.")

    df['returns'] = np.log(df['close'] / df['close'].shift(1))
    df['strategy'] = df['signal'].shift(1) * df['returns']
    df['creturns'] = df['returns'].cumsum().apply(np.exp)
    df['cstrategy'] = df['strategy'].cumsum().apply(np.exp)
    perf = df['cstrategy'].iloc[-1]
    buy_and_hold = df['creturns'].iloc[-1]
    outperf = perf - buy_and_hold
    return round(perf, 6), round(buy_and_hold, 6), round(outperf, 6)

def optimize_super_trend_parameters(df, multipliers, periods):
    best_performance = -np.inf
    best_params = None
    for multiplier, period in product(multipliers, periods):
        temp_df = df.copy()
        temp_df=calculate_super_trend(temp_df, multiplier, period)
        performance = calculate_super_trend_performance(temp_df)[0]
        if performance > best_performance:
            best_performance = performance
            best_params = (multiplier, period)
    return best_params

def calculate_profit_percentage(df):
    initial_principal = 100000  # Start with $100,000
    cash = initial_principal
    shares = 0

    for index, row in df.iterrows():
        if row['Buy/Sell'] == 'Buy' and cash > 0:
            shares_bought = cash // row['close']
            shares += shares_bought
            cash -= shares_bought * row['close']
        elif row['Buy/Sell'] == 'Sell' and shares > 0:
            cash += shares * row['close']
            shares = 0

    # If there are any shares left, sell them at the last recorded price
    if shares > 0:
        cash += shares * df['close'].iloc[-1]
        shares = 0

    final_principal = cash
    profit = final_principal - initial_principal
    profit_percentage = (profit / initial_principal) * 100
    return profit_percentage

def calculate_trade_percentages(df):
    # Ensure the dataframe contains 'Buy/Sell' column
    if 'Buy/Sell' not in df.columns:
        raise ValueError("Buy/Sell column not created. Check calculate_super_trend function.")
    
    # Lists to store buy and sell prices
    buy_prices = []
    sell_prices = []

    # Track the latest close value when the latest signal is 'Buy'
    latest_close = df.tail(1)['close'].values[0]  # Get the latest close value

    # Loop through the dataframe to collect buy and sell prices
    for index, row in df.iterrows():
        if row['Buy/Sell'] == 'Buy':
            buy_prices.append(row['close'])
        elif row['Buy/Sell'] == 'Sell' and buy_prices:
            sell_prices.append(row['close'])

    # Handle the case where the latest signal is 'Buy'
    if len(buy_prices) > len(sell_prices):
        sell_prices.append(latest_close)

    # Calculate percentage changes for each trade
    trade_percentages = [(sell - buy) / buy * 100 for buy, sell in zip(buy_prices, sell_prices)]

    # Separate percentage changes into success and loss
    success_percentages = [p for p in trade_percentages if p > 0]
    loss_percentages = [p for p in trade_percentages if p < 0]

    # Calculate average success and loss percentages
    average_success_percentage = np.mean(success_percentages) if success_percentages else 0
    average_loss_percentage = np.mean(loss_percentages) if loss_percentages else 0

    return average_success_percentage, average_loss_percentage


def get_market_cap(symbol):
    def fetch_market_cap(ticker_symbol):
        try:
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            market_cap = info.get('marketCap')
            return market_cap
        except Exception:
            return None

    market_cap = fetch_market_cap(symbol + '.NS')

    if market_cap is None:
        market_cap = fetch_market_cap(symbol + '.BO')

    if market_cap is None:
        return -1

    # Convert market cap to crores (1 crore = 10 million)
    market_cap_in_crores = market_cap / 10**7
    return market_cap_in_crores



In [2]:
stocks_df=pd.read_csv('STOCKS_1.csv')
stocks_df = stocks_df.dropna(subset=['Symbol'])
stocks=stocks_df['Symbol']

In [3]:
from tqdm import tqdm
all_stocks_data = []  # Initialize an empty list to store individual stock data
total_stocks = len(stocks)
failed_stocks=[]
success_stocks=[]

multipliers = np.arange(1.5, 3.5, 0.5)
periods = np.arange(7, 22, 1)

for stock in tqdm(stocks, desc="Fetching data", unit="stock"):
    try:
        df,symbol = DataFetcher(stock)
        best_params = optimize_super_trend_parameters(df.copy(), multipliers, periods)
        temp_df = calculate_super_trend(df, best_params[0], best_params[1])
        profit_percentage = calculate_profit_percentage(temp_df)
        average_trade_percentage = calculate_trade_percentages(temp_df)
        all_stocks_data.append({'symbol': symbol, 'multiplier': best_params[0], 'period': best_params[1],'average_success_percentage' : average_trade_percentage[0],'average_loss_percentage' : average_trade_percentage[1], 'percentage': profit_percentage, 'Market Cap cr' : get_market_cap(stock)})
        success_stocks.append(stock)
    except Exception as e:
        failed_stocks.append(stock)
        continue

all_stocks_df = pd.DataFrame(all_stocks_data)  # Concatenate all stock dataframes


Fetching data:  20%|██        | 455/2227 [1:14:14<4:11:08,  8.50s/stock]
1 Failed download:
['NIFTYCONSURDURBL.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
Fetching data:  27%|██▋       | 603/2227 [1:42:23<7:58:35, 17.68s/stock]
1 Failed download:
['NIFTYFINSERVICE.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
Fetching data:  34%|███▎      | 747/2227 [2:06:13<3:50:44,  9.35s/stock]
1 Failed download:
['BANKNIFTY.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
Fetching data:  34%|███▍      | 764/2227 [2:08:43<2:41:33,  6.63s/stock]
1 Failed download:
['NIFTYMIDSELECT.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
Fetching data:  35%|███▍      | 778/2227 [2:11:11<3:29:12,  8.66s/stock]
1 Failed download:
['CNXSMALLCAP.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')
Fetching data:  36%|███▋      | 808/2227 [2:16:15<2:35:11,  6.56s/stock]
1 Failed download:
['NIFTYMIDCAP

In [4]:
csv_file_path = 'Paramns_stocks.csv'
all_stocks_df.to_csv(csv_file_path, index=False)